# Deep Learning Lab: ECE-00450107
## Preparation for Meeting 1: Neural Networks

Before running the code in this file, make sure that you are **activating the enviourment** in which the following packages are installed.

#### Definitions and Imports:

In [ ]:
####################################
## DO NOT EDIT THIS CODE SECTION
from DL_Lab1_functions import *
warnings.filterwarnings('ignore')
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
matplotlib.use('Agg')

%matplotlib tk
####################################

#### Set fixed seeds to enable reproducing the results:

In [ ]:
####################################
## DO NOT EDIT THIS CODE SECTION
seed = 50 # age of SIPL :-)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.use_deterministic_algorithms(True)
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.benchmark = False
####################################

## Example for a Fully Connected Neural Classifier

Here we will build and train from scratch a fully-connected neural classifier on the MNIST  dataset.
<center width="100%"><img src="./assets/mnist.jpeg" width="300px"></center>


Check if computer uses CPU or GPU

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Load MNIST dataset and get to know it

In [ ]:
train = torchvision.datasets.MNIST(root="/usr/share/DL_exp/datasets/mnist", train=True, download=True)
test = torchvision.datasets.MNIST(root="/usr/share/DL_exp/datasets/mnist", train=False, download=True)
train_size = len(train)
test_size = len(test)

# Display a few samples
display_Mnist_Sample(train)

Save data in tensors

In [ ]:
train_data = train.data.to(torch.float).to(device)
train_labels = train.targets.to(device)
test_data = test.data.to(torch.float).to(device)
test_labels = test.targets.to(device)

Normalize the images in the training dataset (standardize them) using the well known formula:
$$Z = \frac{X-\mu_X}{\sigma_X}$$

In [ ]:
# get the mean and the std of the train set
train_mean = torch.mean(train_data)
train_std = torch.std(train_data)

# normalize the train and test sets using the above mean and std
train_data = (train_data-train_mean)/train_std
test_data = (test_data-train_mean)/train_std

Define the class of the Neural Network

In [ ]:
# Creating a Fully Connected Neural Network Architeture Class 
class OurNetwork(nn.Module):
    def __init__(self, input_size: int, hidden_layer_size: int, output_size: int):
        super(OurNetwork, self).__init__()
        # define the network fully connected layers
        self.fc_layer1 = nn.Linear(input_size, hidden_layer_size)
        self.fc_layer2 = nn.Linear(hidden_layer_size, output_size)
        # define a flatten layer
        self.flatten = nn.Flatten()
        # define a sigmoid (activation) layer
        self.activation = nn.Sigmoid()
      
    def forward(self, x):
        # define the input layer, operating on flattaned inputs
        flattened_x = self.flatten(x)
        # define the first layer, using linear operation and then activation
        z1 = self.fc_layer1(flattened_x)
        z2 = self.activation(z1)
        # define the output layer
        return self.fc_layer2(z2)

Define some hyper-parameters that will be used during the training.

In [ ]:
# Defining hyper-parameters
hparams = Hyper_Params()
hparams.train_size = train_size
hparams.lr = 0.001 
hparams.batch_size = 1000
hparams.epochs = 30

Set up the model, and define Gradient Descend as optimizer and Cross Entropy for loss.

In [ ]:
# Set the network model with a hidden layer of size 200 and send to device
model = OurNetwork(input_size=784, hidden_layer_size=200, output_size=10).to(device)

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=hparams.lr)
# Define the loss criterion
loss_function = nn.CrossEntropyLoss()

Train the Model

In [ ]:
# Start a progress graph and a performance table, for visualization of the trainig process:
hparams.fig, (hparams.ax1, hparams.ax2) = plt.subplots(2, 1, figsize=(15, 9))
print_performance_grid(Flag=True)
# Calculate how many iterations the model trains in each epoch
iter_num = int(np.ceil(hparams.train_size/hparams.batch_size))

# Set the model to training mode
model.train()
start_time = time.time() # time the start of training
# Training loop:
for epoch in range(hparams.epochs):
    # for each epoch, do:
    hparams.epoch_accuracy_train = np.zeros(iter_num)
    hparams.epoch_loss_train = np.zeros(iter_num)
    # randomly reshuffle the training and test groups before each new epoch:
    index = torch.randperm(hparams.train_size)
    train_data_perm = train_data[index]
    train_labels_perm = train_labels[index]
    # for each batch, do:
    for i, batch in enumerate(range(0, hparams.train_size, hparams.batch_size)):
        # Get a new batch of images and labels
        data = train_data_perm[batch:batch+hparams.batch_size].to(device) 
        target = train_labels_perm[batch:batch+hparams.batch_size].squeeze().to(device)
        
        # Forward pass
        output = model(data) # Apply the network on the new examples
        loss = loss_function(output, target) # calculate the value of the loss function
        
        # Backward pass - ALWAYS IN THIS ORDER!
        optimizer.zero_grad() # First, delete the gradients from the previous iteration
        loss.backward() # Run backward pass on the loss
        optimizer.step() # Preform an algorithm step (using the optimizer)

        # Save the loss and accuracy for the graph visualization
        hparams.epoch_accuracy_train[i] = multi_class_accuracy(output, target.squeeze().to(device))
        hparams.epoch_loss_train[i] = loss.item()
        if(i == 0 and epoch == 0) or ((i+1) == iter_num):
            # Freeze the model in order to evaluate the loss and accuracy on the test set
            model.eval()
            test_out = model(test_data)
            test_loss = loss_function(test_out, test_labels.squeeze()).item()
            test_accuracy = multi_class_accuracy(test_out, test_labels.squeeze())
            print_performance(epoch, i, hparams, test_loss, test_accuracy)
            model.train()
                      
plt.show()
print(f"Training runs with {device}")
print(f"Total training took {time.time() - start_time:.2f} seconds")